# Build
---
This notebook builds Cirrus. It is mainly intended for use by Cirrus developers to push a new version of the software.

## Setup
---

In [1]:
# To ease development, each time a cell is run, all modules will be reloaded.
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import sys
import atexit
import subprocess

In [3]:
# Cirrus produces logs. Here we enable printing them in the notebook
from cirrus import utilities
utilities.set_logging_handler()

/home/tkice/anaconda3/envs/cirrus_env_2.7/lib/python2.7/site-packages/cryptography/__init__.py:39: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in a future release.
  CryptographyDeprecationWarning,


[     _initialize | ResourceManager] Initializing Lambda client.
[     _initialize | ResourceManager] Initializing no-retries Lambda client.
[     _initialize | ResourceManager] Initializing IAM resource.
[     _initialize | ResourceManager] Initializing EC2 resource.
[     _initialize | ResourceManager] Initializing Cloudwatch Logs client.
[     _initialize | ResourceManager] Initializing S3 resource.
[     _initialize | ResourceManager] Initializing STS client.


In [4]:
from cirrus import automate
from cirrus import setup

## Ubuntu (parameter server)
---
Cirrus parameter server is optimized for Ubuntu. In this section we create an executable for the parameter server.

### Build image
First, we create a "build image", which is an AMI on which we've set up the proper environment for building Cirrus. Making the build image is separated from making the executables because the build image will rarely need to be remade, whereas the executables will need to be remade for each new version of the system.

In [6]:
automate.make_ubuntu_build_image("cirrus_ubuntu_build_image")

[make_ubuntu_build_image |      MainThread] Deleting any existing images with the same name.
[   delete_images |      MainThread] Describing images.
[   delete_images |      MainThread] Deleting image ami-07de2cca76f293313.
[   delete_images |      MainThread] Done.
[make_ubuntu_build_image |      MainThread] Launching an instance.
[        __init__ |      MainThread] Done.
[         _exists |      MainThread] Listing instances.
[         _exists |      MainThread] No existing instance with the same name was found.
[ _start_and_wait |      MainThread] Starting a new instance.
[ _start_and_wait |      MainThread] Waiting for instance to enter running state.


KeyboardInterrupt: 

### Executables
Second, we compile the Cirrus parameter server (using the build image) and publish its executables.

The executables are published to the `cirrus-public` S3 bucket.

In [7]:
automate.make_executables(
    setup.PUBLISHED_BUILD + "/executables/ubuntu",
    ("self", "cirrus_ubuntu_build_image"),
    "ubuntu"
)

[make_executables |      MainThread] Launching an instance.
[        __init__ |      MainThread] Resolving AMI owner/name to AMI ID.


RuntimeError: No AMIs with the given owner/name were found.

## Amazon Linux (Lambdas)
---
AWS Lambdas run on an Amazon Linux image. Here we build the Cirrus binaries that runs on those AWS Lambdas.

### Build image
Here we create an image used to compile the Cirrus workers executables.

In [ ]:
automate.make_amazon_build_image("cirrus_amazon_build_image")

[make_amazon_build_image |      MainThread] Deleting any existing images with the same name.
[   delete_images |      MainThread] Describing images.
[   delete_images |      MainThread] Deleting image ami-0677ce11803b154f5.
[   delete_images |      MainThread] Done.
[make_amazon_build_image |      MainThread] Launching an instance.
[        __init__ |      MainThread] Done.
[         _exists |      MainThread] Listing instances.
[         _exists |      MainThread] No existing instance with the same name was found.
[ _start_and_wait |      MainThread] Starting a new instance.
[ _start_and_wait |      MainThread] Waiting for instance to enter running state.
[ _start_and_wait |      MainThread] Fetching instance metadata.
[ _start_and_wait |      MainThread] Done.
[           start |      MainThread] Done.
[make_amazon_build_image |      MainThread] Setting up the environment.
[     run_command |      MainThread] Calling _connect_ssh.
[    _connect_ssh |      MainThread] Configuring.
[  

### Executables
Here we compile the lambda workers executables.

In [12]:
automate.make_executables(
    setup.PUBLISHED_BUILD + "/executables/amazon",
    ("self", "cirrus_amazon_build_image"),
    "ec2-user"
)

[make_executables |      MainThread] Launching an instance.
[make_executables |      MainThread] Launching an instance.
[        __init__ |      MainThread] Resolving AMI owner/name to AMI ID.
[        __init__ |      MainThread] Resolving AMI owner/name to AMI ID.
[        __init__ |      MainThread] Done.
[        __init__ |      MainThread] Done.
[         _exists |      MainThread] Listing instances.
[         _exists |      MainThread] Listing instances.
[         _exists |      MainThread] No existing instance with the same name was found.
[         _exists |      MainThread] No existing instance with the same name was found.
[ _start_and_wait |      MainThread] Starting a new instance.
[ _start_and_wait |      MainThread] Starting a new instance.
[ _start_and_wait |      MainThread] Waiting for instance to enter running state.
[ _start_and_wait |      MainThread] Waiting for instance to enter running state.


KeyboardInterrupt: 

# Cleanup
---
If a cell errors, running this should clean up any resources that were created. After running this cell, the kernel will become unusable and need to be restarted.

In [ ]:
atexit._run_exitfuncs()